# Loading a Model

In [1]:
import os
from os import path
import zipfile
import re  # RegEx
import string
import tensorflow as tf

In [2]:
os.listdir('.')

['.config', 'text_classification_model', 'model.zip', 'sample_data']

In [3]:
model_name = 'text_classification_model'
#
if not path.exists(model_name):
  # Unzip the model
  path_to_zip_file = "model.zip"
  directory_to_extract_to = "."
  with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)
else:
  print("Model has already been unzipped")

Model has already been unzipped


In [4]:
model_path = os.path.join('.', model_name)
model_content = os.listdir(model_path) 

print("Model content {}".format(model_content))

Model content ['saved_model.pb', 'variables']


In [5]:
print("TensorFlow version {}".format(tf.__version__))

TensorFlow version 2.3.0


### Now load the model

In [6]:
@tf.keras.utils.register_keras_serializable(
    package='Custom', name=None
)
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')

model = None
with tf.keras.utils.custom_object_scope({'custom_standardization': custom_standardization}):
  model = tf.keras.models.load_model('text_classification_model')  # Folder

if model is not None:  
  model.summary()
else:
  print("Model not loaded, sorry...")  


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization_1 (TextVe (None, 250)               0         
_________________________________________________________________
sequential (Sequential)      (None, 1)                 160033    
_________________________________________________________________
activation (Activation)      (None, 1)                 0         
Total params: 160,033
Trainable params: 160,033
Non-trainable params: 0
_________________________________________________________________


## Making predictions with the reconstructed model

In [7]:
examples = [
  "The movie was great!",
  "The movie was okay.",
  "The movie was terrible...",
  "That thing really sucks",
  "I kind of liked it",
  "Never seen a better one",
  "Not really good, not really bad, in my opinion.",
  "Not really gorgeous"
]

predictions = model.predict(examples)
print("-- Predictions --")
for i in range(len(examples)):
    comment = examples[i]
    score = predictions[i][0]
    review = "So-so"
    if score < 0.39:
        review = "Very negative review"
    elif score < 0.41:
        review = "Negative review"
    elif score > 0.5:
        review = "Positive review"
    print("{} => {} : {}".format(comment, score, review))
print("-------------------")

-- Predictions --
The movie was great! => 0.6212304830551147 : Positive review
The movie was okay. => 0.4448379576206207 : So-so
The movie was terrible... => 0.3627158999443054 : Very negative review
That thing really sucks => 0.3784532845020294 : Very negative review
I kind of liked it => 0.60201096534729 : Positive review
Never seen a better one => 0.550157904624939 : Positive review
Not really good, not really bad, in my opinion. => 0.4071146845817566 : Negative review
Not really gorgeous => 0.5238542556762695 : Positive review
-------------------


### Done!